In [0]:
import tensorflow as tf
tf.enable_eager_execution()

In [0]:
import numpy as np
import os
import time

In [0]:
# reading file

text = open('shakespeare.txt').read()
print('Total characters: %d' %(len(text)))

Total characters: 1115394


In [0]:
print(text[:250])
# text = text[:10000]

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [0]:
# extracting unique elements in the text

vocab = sorted(list(set(text)))
print('Total vocab: %d' %(len(vocab)))

Total vocab: 65


In [0]:
# a numpy series containing all the unique elements

int_to_char = np.array(vocab)
print(int_to_char)
# print()
# print(int_to_char.shape) # (65,)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [0]:
# a dict pointing char to index

char_to_int = dict((ch,i) for i,ch in enumerate(vocab))
char_to_int

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

In [0]:
# converting input format from char to int

text_as_int = np.array([char_to_int[ch] for ch in text])
# text_as_int.shape # (1115394,)

In [0]:
text_as_int [:100]

array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43,
       44, 53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39,
       52, 63,  1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1,
       51, 43,  1, 57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31,
       54, 43, 39, 49,  6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56,
       57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [0]:
# Tensors can be explicitly converted to NumPy ndarrays by invoking the .numpy() method on them

seq_length = 100 # number of time-steps; length of each example
examples_per_epoch = len(text)//seq_length  # 11153; number of sequences/examples per epoch

# tf.data.Dataset.from_tensor_slices function converts numpy dataset into tensorflow dataset
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for char in char_dataset.take(5):
    print(int_to_char[char.numpy()])

F
i
r
s
t


In [0]:
# type(char_dataset) # tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

In [0]:
# dividing text into sequences/examples each of size = 101
# input : 0-100 ; output : 1-101

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for char in sequences.take(3):
    print(repr(''.join(int_to_char[char.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"


In [0]:
# split each sequence into input[0-100] and target[1-101]

def split_into_input_target(chunk):
    inputs = chunk[:-1]
    targets = chunk[1:]
    return inputs, targets

dataset = sequences.map(split_into_input_target)

In [0]:
for inputs, targets in dataset.take(1):
    print('Input data:', repr(''.join(int_to_char[inputs.numpy()])))
    print('Target data:', repr(''.join(int_to_char[targets.numpy()])))

Input data: 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [0]:
for i, (input_idx, target_idx) in enumerate(zip(inputs[:5], targets[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(int_to_char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(int_to_char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [0]:
# examples_per_epoch # 11153

# Create training batches

In [0]:
# Batch size

batch_size = 64
steps_per_epoch = examples_per_epoch//batch_size # no. of batches per epoch ; 174

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements)
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)
dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

# Build the Model

In [0]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [0]:
if tf.test.is_gpu_available():
    rnn = tf.keras.layers.CuDNNGRU
else:
    import functools
    rnn = functools.partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        rnn(rnn_units,
            return_sequences=True,
            recurrent_initializer='glorot_uniform',
            stateful=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)

# Try the model

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
    examples_batch_predictions = model(input_example_batch)
    print(examples_batch_predictions.shape, '# (batch_size, sequence_length, vocab_size)')

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3935232   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,018,497
Trainable params: 4,018,497
Non-trainable params: 0
_________________________________________________________________


In [0]:
# For the first example in the batch

sample_indices = tf.random.categorical(examples_batch_predictions[0], num_samples=1)

# Given a tensor input, 'tf.squeeze' returns a tensor of the same type
# with all dimensions of size 1 removed. If you don't want to remove all
# size 1 dimensions, you can remove specific size 1 dimensions by specifying axis
# e.g.- here last tensor is being removed

# 't' is a tensor of shape [1, 2, 1, 3, 1, 1]; tf.shape(tf.squeeze(t))  # [2, 3]
# 't' is a tensor of shape [1, 2, 1, 3, 1, 1]; tf.shape(tf.squeeze(t, [2, 4]))  # [1, 2, 3, 1]
# 't' is a tensor of shape [1, 2, 1, 3, 1, 1]; tf.shape(tf.squeeze(t, [-1]))  # [1, 2, 1, 3, 1]
sample_indices = tf.squeeze(sample_indices, axis=-1).numpy()

In [0]:
# This gives us, at each timestep, a prediction of the next character index:

sample_indices # (100, )

array([10, 52, 24, 61, 43, 24,  9, 31, 60,  7, 61,  2, 20,  1, 34,  4, 34,
       46, 54, 42, 38,  6, 40, 59, 12,  1, 43,  7, 47,  6,  6, 11, 39, 48,
       51,  7, 11, 45, 44,  0, 23, 41, 43, 58, 62, 14, 26, 40, 54, 42, 44,
       36, 55, 57, 47, 64, 38, 20, 28,  7, 35, 41, 54,  8, 52,  7,  1, 19,
       37, 55, 34, 51, 41, 54, 20, 22,  4, 38, 40, 29, 21, 61, 55, 22, 58,
       44, 41,  7,  9, 37,  5, 15, 24,  4, 48, 35, 29, 28, 22, 45])

In [0]:
print("Input:", repr("".join(int_to_char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(int_to_char[sample_indices])))

Input: "uried in the king's highway,\nSome way of common trade, where subjects' feet\nMay hourly trample on th"

Next Char Predictions: 
 ":nLweL3Sv-w!H V&VhpdZ,bu? e-i,,;ajm-;gf\nKcetxBNbpdfXqsizZHP-Wcp.n- GYqVmcpHJ&ZbQIwqJtfc-3Y'CL&jWQPJg"


# Train the model

In [0]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
  
example_batch_loss = loss(target_example_batch, examples_batch_predictions)
print('Prediction shape: ', example_batch_loss.shape, ' # (batch_size, sequence_length)')
print('scalar_loss: ',example_batch_loss.numpy().mean())

Prediction shape:  (64, 100)  # (batch_size, sequence_length)
scalar_loss:  4.1745753


In [0]:
model.compile(optimizer=tf.train.AdamOptimizer(), loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = 'training_checkpoints'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True)

In [30]:
# steps_per_epoch = 174
epochs = 5
history = model.fit(dataset.repeat(), epochs=epochs, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/5
174/174 [==============================] - 1729s 10s/step - loss: 2.6763
Epoch 2/5
174/174 [==============================] - 1717s 10s/step - loss: 1.9334
Epoch 3/5
174/174 [==============================] - 1716s 10s/step - loss: 1.6793
Epoch 4/5
174/174 [==============================] - 1721s 10s/step - loss: 1.5377
Epoch 5/5
174/174 [==============================] - 1729s 10s/step - loss: 2.6763
Epoch 2/5
174/174 [==============================] - 1717s 10s/step - loss: 1.9334
Epoch 3/5
174/174 [==============================] - 1716s 10s/step - loss: 1.6793
Epoch 4/5
174/174 [==============================] - 1721s 10s/step - loss: 1.5377
Epoch 5/5
174/174 [==============================] - 1735s 10s/step - loss: 1.4495


# Generate text

In [31]:
tf.train.latest_checkpoint(checkpoint_dir)

'training_checkpoints/ckpt_5'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,018,497
Trainable params: 4,018,497
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
    # Number of chars to generate
    num_generate = 1000
    
    # Converting our start string to numbers (vectorizing)
    input_eval = [char_to_int[s] for s in start_string] # a numpy list
#     print(input_eval) # [30, 27, 25, 17, 27, 10, 1]
    
    # Given a tensor input, this operation inserts a dimension 
    # of 1 at the dimension index axis of input's shape.
    input_eval = tf.expand_dims(input_eval, 0) # a tensor
#     print(input_eval.numpy()) # [[30 27 25 17 27 10  1]]
#     print(input_eval.shape) # (1, 7)
    
    # Empty string to store our results
    text_generated = []
    
    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    temperature = 1.0
    
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
#         print(predictions.numpy())
#         print(predictions.shape) # (1, 7, 65)
        
        # Remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
#         print(predictions.numpy())
#         print(predictions.shape) # (7, 65)
        
        # Using a multinomial distribution to predict the word returned by the model
        predictions = predictions / temperature
#         print(tf.multinomial(predictions, num_samples=1).numpy())
        predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
#         print(predicted_id) # 20
#         print(predicted_id.shape) # () ----> scalar
        
        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
#         print(input_eval.numpy()) # [[20]]
#         print(input_eval.shape) # (1, 1)
        
#         print(int_to_char[predicted_id]) # H
        text_generated.append(int_to_char[predicted_id])
        
    return (start_string + ''.join(text_generated))

In [35]:
# print(generate_text(model, start_string=u"ROMEO: "))
print(generate_text(model, start_string="ROMEO: "))

W0704 08:35:49.018097 140170663364480 deprecation.py:323] From <ipython-input-34-105c652492eb>:36: multinomial (from tensorflow.python.ops.random_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.random.categorical` instead.


ROMEO: I was
Maken him kingrish'd as dnown: but this is the viely sour!

Veiden: I'll tell thee her frows, to your hermits, and the till should ring by
it in that ay, thereon night, my lord.
The varat! But too lay him,
You know my strusk is one fair point: I do gan us yet?

HENRY BOLINGBROKE:
None watching others. Do much many masters, and thou dast
You: good my bose before his swift,
From whom to yea, or now to fribe,
Or all man, do not sighs for a merry,
Who's tho gaves door majesty, till thee:
I soonet dot?

GEORTES:
I saws and cause, depair;
For newnresed to her hagry in the crown.

KING EDWARD II:
Adswer to fash, to of thees with him.
I pray you, my anghy day.
You knowly confess, quite to find
o'e my love's mather flostly and but thee.

ALO:
I take your late most officer:
That speak again. I say, my chapter, to the chird?

Numbert:
O, I shall but concees to seals make the postring plack my love.
I they are noble and upon the walls? ip to thine in purcus.

MASCIUS:
No, but I'll see